In [12]:
# from google.colab import drive
# drive.mount('/content/drive')

In [13]:
# !pip install ultralytics -q

In [14]:
from ultralytics import YOLO
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import math

In [15]:
image_path = "/content/drive/MyDrive/HawkEyes/japan/bolt/Image__2024-03-27__13-43-41.png"

In [16]:
pose_model = YOLO("/content/drive/MyDrive/HawkEyes/japan/pose/best.pt")
obb_model = YOLO("/content/drive/MyDrive/HawkEyes/japan/obb/best1.pt")

In [17]:
pose_predict = pose_model.predict(image_path , save = True , save_txt = True)
obb_predict = obb_model.predict(image_path , save = True , save_txt = True)
pose_points = pose_predict[0].keypoints.xy[0].numpy().astype(int)
obb_points = obb_predict[0].obb.xyxyxyxy[0].numpy().astype(int)


image 1/1 /content/drive/MyDrive/HawkEyes/japan/bolt/Image__2024-03-27__13-43-41.png: 480x640 1 Bolt, 193.8ms
Speed: 4.0ms preprocess, 193.8ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/pose/predict2
1 label saved to runs/pose/predict2/labels

image 1/1 /content/drive/MyDrive/HawkEyes/japan/bolt/Image__2024-03-27__13-43-41.png: 480x640 285.9ms
Speed: 5.4ms preprocess, 285.9ms inference, 2.6ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/obb/predict2
1 label saved to runs/obb/predict2/labels


In [18]:
img = cv2.imread(image_path)

# Find Top Mid point
xmidtop = int((obb_points[0][0] + obb_points[1][0]) / 2)
ymidtop = int((obb_points[0][1] + obb_points[1][1]) / 2)

# Find Bottom Mid point
xmidbot = int((obb_points[2][0] + obb_points[3][0]) / 2)
ymidbot = int((obb_points[2][1] + obb_points[3][1]) / 2)

# Find Left Mid point
xmidlft = int((obb_points[0][0] + obb_points[3][0]) / 2)
ymidlft = int((obb_points[0][1] + obb_points[3][1]) / 2)

# Find Right Mid point
xmidrit = int((obb_points[1][0] + obb_points[2][0]) / 2)
ymidrit = int((obb_points[1][1] + obb_points[2][1]) / 2)

# Find Center Mid point
xmidmid = int((xmidtop + xmidbot) / 2)
ymidmid = int((ymidtop + ymidbot) / 2)

# Calculate ManHattan Distance From POSE Head
p1 = int (abs(pose_points[0][0]-obb_points[0][0]))
q1 = int (abs(pose_points[0][1]-obb_points[0][1]))
p2 = int (abs(pose_points[0][0]-obb_points[3][0]))
q2 = int (abs(pose_points[0][1]-obb_points[3][1]))
d1 = p1+q1
d2 = p2+q2

if d1<d2:
  cx = xmidtop
  cy = ymidtop

else:
  cx = xmidbot
  cy = ymidbot

cv2.polylines(img, [obb_points], isClosed=True, color=(255, 255, 0), thickness=1) # Draw OBB Boundry
cv2.line(img, (xmidtop, ymidtop), (xmidbot, ymidbot), (0, 255, 0), 2) # Object Vertical Line
cv2.line(img, (xmidlft, ymidlft), (xmidrit, ymidrit), (0, 255, 0), 2) # Object Horizontal Line
cv2.line(img, (obb_points[0][0]-100, ymidmid), (obb_points[2][0]+100, ymidmid), (20, 20, 20), 2) # Vertical Line
cv2.line(img, (xmidmid, obb_points[0][1]), (xmidmid, obb_points[2][1]), (20, 20, 20), 2) # Horizontal Line
cv2.circle(img, (pose_points[0][0],pose_points[0][1]), radius=5, color=(0, 0, 255), thickness=-1) # Head
cv2.circle(img, (cx,cy), radius=5, color=(255, 0, 255), thickness=-1) # Head Mid pink
cv2_imshow(img)

Output hidden; open in https://colab.research.google.com to view.

In [19]:
import math

def line_angle(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.atan2(dy, dx)

angle = -1*(math.degrees(line_angle(xmidmid, ymidmid, cx, cy)))

if angle<0:
  angle = 360+angle

print("Angle of the line:", angle)

Angle of the line: 229.23061260314415
